In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
stats = pd.read_csv('cleaned_fbref_players.csv')

/var/folders/qt/m1d2k3qd4_z368vlhrz0r7w80000gn/T/ipykernel_78831/3895760055.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  stats = pd.read_csv('cleaned_fbref_players.csv')


In [ ]:
num_cols = stats.select_dtypes(include='number').columns.drop('Min_Playing')

def agg_per_url(g):
    total_minutes = g['Min_Playing'].sum()
    # weighted averages
    out = {
        col: (g[col] * g['Min_Playing']).sum() / total_minutes
        for col in num_cols
    }
    out['Total_Min_Playing'] = total_minutes
    # carry along Player/Nation/Pos
    out['Player'] = g['Player'].iloc[0]
    out['Nation'] = g['Nation'].iloc[0]
    out['Pos']    = g['Pos'].iloc[0]
    return pd.Series(out)

# group and apply
weighted_by_url = stats.groupby('Url').apply(agg_per_url).reset_index()

In [ ]:
front_cols = ['Url', 'Player', 'Nation', 'Pos']

other_cols = [c for c in weighted_by_url.columns if c not in front_cols]

weighted_by_url = weighted_by_url[ front_cols + other_cols ]

In [ ]:
# Load your RAPM ratings CSV
ratings = pd.read_csv('baseline_model.csv')

ratings = ratings[['player_name','offense']]

ratings = ratings.rename(columns={
    'player_name': 'Player',
    'offense':     'offense_rating'
})

# Merge onto weighted_by_url
merged_df = weighted_by_url.merge(
    ratings,
    on='Player',
    how='left'           
)